In [70]:
import pandas as pd
import sqlite3
from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings("ignore")

In [71]:
db = sqlite3.connect('../data/referrals.sqlite')

for chunk in tqdm(pd.read_csv('../data/doc_graph.csv', chunksize = 10000)): #Reading data set in by chunks
    chunk.columns = [x.lower().replace(' ', '_') for x in chunk.columns] #making columns SQL format
    chunk.to_sql('doc_graph', db, if_exists = 'append', index = False) #building the database

0it [00:00, ?it/s]

In [72]:
db.close()

In [73]:
#These are the only needed columns. Stll some cleaning to do.
needed_columns = ['NPI', 
    'Entity Type Code',
    'Provider Organization Name (Legal Business Name)',
    'Provider Last Name (Legal Name)',
    'Provider First Name',
    'Provider Middle Name',
    'Provider Name Prefix Text',
    'Provider Name Suffix Text',
    'Provider Credential Text',
    'Provider First Line Business Practice Location Address',
    'Provider Second Line Business Practice Location Address',
    'Provider Business Practice Location Address City Name',
    'Provider Business Practice Location Address State Name',
    'Provider Business Practice Location Address Postal Code',
    'Healthcare Provider Taxonomy Code_1',
    'Healthcare Provider Taxonomy Code_2',
    'Healthcare Provider Taxonomy Code_3',
    'Healthcare Provider Taxonomy Code_4',
    'Healthcare Provider Taxonomy Code_5',
    'Healthcare Provider Taxonomy Code_6',
    'Healthcare Provider Taxonomy Code_7',
    'Healthcare Provider Taxonomy Code_8',
    'Healthcare Provider Taxonomy Code_9',
    'Healthcare Provider Taxonomy Code_10',
    'Healthcare Provider Taxonomy Code_11',
    'Healthcare Provider Taxonomy Code_12',
    'Healthcare Provider Taxonomy Code_13',
    'Healthcare Provider Taxonomy Code_14',
    'Healthcare Provider Taxonomy Code_15',
    'Healthcare Provider Primary Taxonomy Switch_1',
    'Healthcare Provider Primary Taxonomy Switch_2',
    'Healthcare Provider Primary Taxonomy Switch_3',
    'Healthcare Provider Primary Taxonomy Switch_4',
    'Healthcare Provider Primary Taxonomy Switch_5',
    'Healthcare Provider Primary Taxonomy Switch_6',
    'Healthcare Provider Primary Taxonomy Switch_7',
    'Healthcare Provider Primary Taxonomy Switch_8',
    'Healthcare Provider Primary Taxonomy Switch_9',
    'Healthcare Provider Primary Taxonomy Switch_10',
    'Healthcare Provider Primary Taxonomy Switch_11',
    'Healthcare Provider Primary Taxonomy Switch_12',
    'Healthcare Provider Primary Taxonomy Switch_13',
    'Healthcare Provider Primary Taxonomy Switch_14',
    'Healthcare Provider Primary Taxonomy Switch_15']

In [74]:
#These are the desried columns after cleaning is finished.
wanted_columns = ['NPI', 
    'Entity Type Code',
    'Provider Organization Name (Legal Business Name)',
    'Provider Last Name (Legal Name)',
    'Provider First Name',
    'Provider Middle Name',
    'Provider Name Prefix Text',
    'Provider Name Suffix Text',
    'Provider Credential Text',
    'Provider First Line Business Practice Location Address',
    'Provider Second Line Business Practice Location Address',
    'Provider Business Practice Location Address City Name',
    'Provider Business Practice Location Address State Name',
    'Provider Business Practice Location Address Postal Code',
    'taxonomy_code']

In [76]:
db = sqlite3.connect('../data/referrals.sqlite')

for chunk in tqdm(pd.read_csv('../data/npi_data_pfile.csv', usecols = needed_columns, chunksize = 10000)): #Reading data in by chuncks
    list_df = [] 

    for i in range (1,16): #Searching for the taxonomy code and creating its own column
        yes = npi_data[
                    npi_data[f'Healthcare Provider Primary Taxonomy Switch_{i}'] == 'Y'
                ][['NPI', f'Healthcare Provider Taxonomy Code_{i}']
                 ].rename(columns={f'Healthcare Provider Taxonomy Code_{i}':'taxonomy_code'})

        list_df.append(yes) 
    
    final = pd.merge(left = npi_data, right = pd.concat(list_df), how = 'left', on = 'NPI')[wanted_columns] #Merging the taxonomy codes back to npi data 
    
    final.columns = [x.lower().replace(' ', '_') for x in final.columns] #cleaning column names
    final.to_sql('npi_data', db, if_exists = 'append', index = False) #building database

0it [00:00, ?it/s]

In [86]:
db.close()

In [96]:
taxonomy = pd.read_csv('../data/nucc_taxonomy.csv', encoding= 'unicode_escape')

taxonomy.columns = [x.lower().replace(' ', '_') for x in taxonomy.columns]

db = sqlite3.connect('../data/referrals.sqlite')

taxonomy.to_sql('taxonomy', db, if_exists = 'append', index = False)

873

In [97]:
db.close()

In [103]:
db = sqlite3.connect('../data/referrals.sqlite')

pd.read_excel('../data/zip_cbsa.xlsx').to_sql('zipcode', db, if_exists = 'append', index = False)

47484

In [104]:
db.close()